# Read in data sets

The `INPUT_LOAD_AND_VRE_FILENAME` refers to a CSV file that contains data with the following columns:
* Timestamp (at hourly level)
* Load on the Island of Amerigo 
* Aggregated solar generation from all proposed PV sites (compiled from SAM)
* Wind generation from proposed site (compiled from SAM)

In [1]:
import os
from openpyxl import load_workbook
import pandas as pd
from datetime import timedelta

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
INTERIM_DATA_DIR = os.path.join(DATA_DIR, 'interim')

INPUT_LOAD_AND_VRE_FILENAME = 'load_and_vre.csv'
FINAL_LOAD_AND_VRE_FILENAME = 'final_load_and_vre.csv' # filename of CSV after adding columns

In [2]:
# all input columns as variables for later use
date_colname = 'date'
orig_load_colname = 'load_before_transmission_mw'
solar_gen_colname = 'solar_mw'
wind_gen_colname = 'wind_mw'

load_and_vre_df = pd.read_csv(os.path.join(RAW_DATA_DIR, INPUT_LOAD_AND_VRE_FILENAME))
input_cols = [date_colname, orig_load_colname, wind_gen_colname, solar_gen_colname]
load_and_vre_df = load_and_vre_df.iloc[:, [i for i in range(len(input_cols))]]
load_and_vre_df.columns = input_cols

#### Column Names
The below lists all of the columns that we'll create from our input data set, listed as variables

In [3]:
# all derived columns as variables for later use

### COMMON - columns needed bt legacy gen, batteries, hydrogen
day_of_month_colname = 'day_of_month'
day_of_yr_colname = 'day_of_yr'
week_of_yr_colname = 'wk_of_yr'
month_colname = 'month'
year_colname = 'year'
hour_colname = 'hour'
load_w_transmission_loss_and_load_scale_colname = 'load_w_transmission_loss_and_load_scale'
total_vre_gen_colname = 'total_vre_gen_mw'
therm_gen_baseline_colname = 'therm_gen_baseline'
load_after_therm_gen_baseline_colname = 'load_after_therm_gen_baseline'
load_after_therm_gen_baseline_and_vre_prod_colname = 'load_after_therm_gen_baseline_and_vre_prod'
curtailment_after_therm_gen_baseline_and_vre_colname = 'curtailment_after_therm_gen_baseline_and_vre' # Post Grid Demand VRE Curtailment

# addtl common cols, tbd on whether we still need
# weekday_colname = 'weekday'
# load_vre_diff_colname = 'load_vre_diff'
# prev_7_load_colname = 'prev_7_load'
# prev_7_load_vre_diff_colname = 'prev_7_load_vre_diff'
# prev_7_vre_gen_colname = 'prev_7_vre_gen'
# critical_load_mw_colname = 'critical_load_mw'
# critical_load_less_vre_colname = 'critical_load_less_vre_mw'
# load_after_thermal_gen_colname = 'load_after_thermal_gen'
# surplus_vre_colname = 'surplus_vre'

### HYDROGEN
req_hydrogen_kg_colname = 'reqd_hydrogen_kg'
hydrogen_production_kg_colname = 'hydrogen_prod_kg'
hydrogen_production_m3_colname = 'hydrogen_prod_m3'
hydrogen_h2o_demand_l_colname = 'hydrogen_h2o_demand_l'
hydrogen_h2o_demand_m3_colname = 'hydrogen_h2o_demand_m3'
compression_desal_demand_mw_colname = 'compression_desal_demand_mw' 
h2_demand_comp_desal_kg_colname = 'h2_demand_comp_desal_kg'
cuml_h2_storage_kg_colname = 'cuml_h2_storage_kg'
cuml_hydrogen_demand_kg_colname = 'cuml_hydrogen_demand_kg'
cuml_h2_energy_demand_mwh_colname = 'cuml_h2_energy_demand_mwh'
h2_opex_usd_per_kg_colname = 'h2_opex_usd_per_kg'
h2_opex_electrolyzer_colname = 'h2_opex_electrolyzer'
electrolyzer_power_input_mw_colname = 'electrolyzer_power_input_mw'
fuel_cell_power_generation_mw_colname = 'fuel_cell_power_generation_mw'

### BAU - LEGACY GENERATION
thermal_gen_total_mw_colname = 'thermal_gen_total_mw'
thermal_gen_rice_mw_colname = 'thermal_gen_rice_mw'
thermal_gen_combustion_mw_colname = 'thermal_gen_combustion_mw'

#### Assumptions

Any variable in all caps represent constants that are either: 
* assumptions based on the literature
* values taken from SAM (e.g., `SOLAR_INSTALLATION_COST_MM`)
* values specified in the assignment PDF

In [4]:
# other values/assumptions needed for the analysis
CRITICAL_LOAD_PERC = (1/3)
SOLAR_SCALE_FACTOR = 1
WIND_SCALE_FACTOR = 1
TRANSMISSION_EFFICIENCY = 1
LOAD_SCALING_FACTOR = 1
THERMAL_BASE_GEN_MW = 31
THERMAL_MIN_GEN_MW = 0

# # CAPEX #############################################
# SOLAR_INSTALLATION_COST_MM = 50.8
# WIND_INSTALLATION_COST_MM = 31.2
# H2_FUEL_CELL_COST_MM = 4.23 
# H2_ELECTROLYZER_COST_MM = 21.12

# # H2 Storage 
# H2_STORAGE_CAPEX_USD_PER_KG = 700 # Parks, G., Boyd, R., Cornish, J., & Remick, R. (2014). Hydrogen Station Compression, Storage, and Dispensing Technical Status and Costs: Systems Integration. Related Information: Independent Review 
# H2_MAX_STORAGE_CAPACITY_KG = 65000
# H2_STORAGE_COST_MM = (H2_STORAGE_CAPEX_USD_PER_KG * H2_MAX_STORAGE_CAPACITY_KG)/ 1_000_000
# # H2 Compression
# H2_COMPRESSION_COST_MM = 7.32
# #####################################################

# # O & M #############################################
# SOLAR_O_AND_M_COST_KW_PER_YR = 9.1 # kW / yr : https://www.nrel.gov/docs/fy19osti/72399.pdf
# WIND_O_AND_M_COST_KW_PER_YR = 44 # kW / yr (ATB - https://atb.nrel.gov/electricity/2019/index.html?t=lw)
# FUEL_CELL_O_AND_M_COST_MM = .05 * H2_FUEL_CELL_COST_MM * 1_000_000 # of capital cost (https://www.nrel.gov/docs/fy16osti/65856.pdf)
# ELECTROLYZER_O_AND_M_COST_KW_PER_YR = 42 #  (https://www.nrel.gov/docs/fy16osti/65856.pdf)
# #####################################################

#### Clean the input data set
The input data set might have `null` values, and we want to make sure the date column is formatted in the way we expect downstream

In [5]:
def clean_input_columns(df):

    cleaned_df = df.copy()
    cleaned_df = cleaned_df.dropna(how='all')

    cleaned_df[date_colname] = pd.to_datetime(cleaned_df[date_colname])
    # all solar vals need to be > 0
    cleaned_df[solar_gen_colname] = cleaned_df[solar_gen_colname]\
        .apply(lambda x: x if x > 0 else 0)
    
    cleaned_df[month_colname] = cleaned_df[date_colname].map(lambda x: x.month)
    cleaned_df[day_of_month_colname] = cleaned_df[date_colname].map(lambda x: x.day)
    cleaned_df[day_of_yr_colname] = cleaned_df[date_colname].map(lambda x: x.timetuple().tm_yday)
    cleaned_df[hour_colname] = cleaned_df[date_colname].map(lambda x: x.hour)
    
    return cleaned_df

In [6]:
cleaned_load_and_vre_df = clean_input_columns(load_and_vre_df)

In [7]:
cleaned_load_and_vre_df

,date,load_before_transmission_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,hour
0,2017-01-01 00:00:00,35.1,1.68,0.0,1,1,1,0
1,2017-01-01 01:00:00,35.1,1.23,0.0,1,1,1,1
2,2017-01-01 02:00:00,34.6,1.09,0.0,1,1,1,2
3,2017-01-01 03:00:00,34.2,1.47,0.0,1,1,1,3
4,2017-01-01 04:00:00,33.8,1.12,0.0,1,1,1,4
...,...,...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,45.5,6.77,0.0,12,31,365,19
8756,2017-12-31 20:00:00,46.5,6.21,0.0,12,31,365,20
8757,2017-12-31 21:00:00,45.0,8.73,0.0,12,31,365,21
8758,2017-12-31 22:00:00,41.7,7.23,0.0,12,31,365,22


#### Assertions
Since the rows of the input data set are expected to represent hours over one year, we'd expect there to be 8760 rows.

Also check that each day only has 24 rows

In [8]:
assert len(cleaned_load_and_vre_df) == 8760, "Expected one row per hour in year (8760), got {} rows".format(len(load_and_gen_df))

for day_of_yr in cleaned_load_and_vre_df[day_of_yr_colname].unique():
    if len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr]) != 24:
        assert ValueError("Day {} has {} values".format(day_of_yr, len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr])))
        

Each of the technologies considered for resilience have some columns in common. This function creates these columns for use in the battery case, business-as-usual case, and hydrogen case

In [9]:
def add_common_derived_columns(df):
    
    decorated_df = df.copy()
    
    decorated_df[solar_gen_colname] = \
        decorated_df[solar_gen_colname] \
        * SOLAR_SCALE_FACTOR
    
    decorated_df[wind_gen_colname] = \
        decorated_df[wind_gen_colname] \
        * WIND_SCALE_FACTOR
        
    decorated_df[load_w_transmission_loss_and_load_scale_colname] = \
        decorated_df[orig_load_colname] / \
        (TRANSMISSION_EFFICIENCY * LOAD_SCALING_FACTOR)

    decorated_df[therm_gen_baseline_colname] = THERMAL_BASE_GEN_MW
    
    decorated_df[load_after_therm_gen_baseline_colname] = \
        decorated_df[load_w_transmission_loss_and_load_scale_colname] \
        - decorated_df[therm_gen_baseline_colname]
    
    decorated_df[total_vre_gen_colname] = \
        decorated_df[wind_gen_colname] + decorated_df[solar_gen_colname]
        
    decorated_df[load_after_therm_gen_baseline_and_vre_prod_colname] = \
        (decorated_df[load_after_therm_gen_baseline_colname] \
        - decorated_df[total_vre_gen_colname]).map(lambda x: x if x > 0 else 0)
    
    decorated_df[curtailment_after_therm_gen_baseline_and_vre_colname] = \
        (decorated_df[total_vre_gen_colname] \
        - decorated_df[load_after_therm_gen_baseline_colname]).map(lambda x: x if x > 0 else 0)

    return decorated_df

In [10]:
final_load_and_vre_df = add_common_derived_columns(cleaned_load_and_vre_df)

# drop the CSV within the project
final_load_and_vre_df.to_csv(os.path.join(INTERIM_DATA_DIR, FINAL_LOAD_AND_VRE_FILENAME), index=False)

In [11]:
final_load_and_vre_df.head(24)

,date,load_before_transmission_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,hour,load_w_transmission_loss_and_load_scale,therm_gen_baseline,load_after_therm_gen_baseline,total_vre_gen_mw,load_after_therm_gen_baseline_and_vre_prod,curtailment_after_therm_gen_baseline_and_vre
0,2017-01-01 00:00:00,35.1,1.68,0.0000,1,1,1,0,35.1,31,4.1,1.6800,2.4200,0.0000
1,2017-01-01 01:00:00,35.1,1.23,0.0000,1,1,1,1,35.1,31,4.1,1.2300,2.8700,0.0000
2,2017-01-01 02:00:00,34.6,1.09,0.0000,1,1,1,2,34.6,31,3.6,1.0900,2.5100,0.0000
3,2017-01-01 03:00:00,34.2,1.47,0.0000,1,1,1,3,34.2,31,3.2,1.4700,1.7300,0.0000
4,2017-01-01 04:00:00,33.8,1.12,0.0000,1,1,1,4,33.8,31,2.8,1.1200,1.6800,0.0000
5,2017-01-01 05:00:00,33.4,0.91,0.0000,1,1,1,5,33.4,31,2.4,0.9100,1.4900,0.0000
6,2017-01-01 06:00:00,33.3,1.84,0.0000,1,1,1,6,33.3,31,2.3,1.8400,0.4600,0.0000
7,2017-01-01 07:00:00,33.8,1.59,6.8978,1,1,1,7,33.8,31,2.8,8.4878,0.0000,5.6878
8,2017-01-01 08:00:00,34.7,1.18,21.3585,1,1,1,8,34.7,31,3.7,22.5385,0.0000,18.8385
9,2017-01-01 09:00:00,34.7,0.78,27.9603,1,1,1,9,34.7,31,3.7,28.7403,0.0000,25.0403


# Battery Analysis

Using the data set above, we can create a battery-specific analysis from the observed data.

This function takes in the hourly data and a config that contains properties of a battery. 

In [12]:
### BATTERIES ################################################################
# intermediate calc columns
battery_soc_colname = 'battery_soc_mwh'
battery_discharge_mw_colname = 'battery_discharge_mw' # Battery Production, MW 
battery_curtailment_after_charge_mw_colname = 'battery_curtailment_after_charge_mw'
thermal_gen_delta_from_baseline_colname = 'thermal_gen_delta_from_baseline'
# optimized case columns

# battery properties that calcs depend on
battery_capacity_mwh_config_prop = "battery_capacity_mwh"
battery_power_mw_config_prop = "battery_power_mw"
battery_efficiency_config_prop = "battery_efficiency"
##############################################################################

# Battery storage ##################################
BATTERY_CAPACITY_MWH = 220
BATTERY_POWER_MW = 40
BATTERY_EFFICIENCY = 0.875
BATTERY_DEPTH_OF_DISCHARGE = 0.5
BATTERY_MIN_SOC_MWH = (1 - BATTERY_DEPTH_OF_DISCHARGE) * BATTERY_CAPACITY_MWH
BATTERY_POWER_KW = BATTERY_POWER_MW * 1000
BATTERY_CAPEX = ((BATTERY_CAPACITY_MWH * 1000 * 260) + (BATTERY_POWER_MW * 1000 * 530))
BATT_O_AND_M_COST_PER_KW_YR = .025 * (BATTERY_CAPEX / BATTERY_POWER_KW) #(https://www.nrel.gov/docs/fy19osti/73222.pdf)
BATTERY_SOC_BEGINNING_OF_YR = 179.91
#####################################################

In [13]:
def add_intermediate_battery_cols(hourly_df, battery_config):
    
    decorated_df = hourly_df.copy()
    
    battery_capacity = battery_config[battery_capacity_mwh_config_prop]
    battery_efficiency = battery_config[battery_efficiency_config_prop]
    
    decorated_df[battery_soc_colname] = None
    decorated_df[battery_curtailment_after_charge_mw_colname] = None
    decorated_df[battery_discharge_mw_colname] = None

    decorated_df.reset_index()
    for idx, row in decorated_df.iterrows():

        if idx == 0: # First row of data set, so can't take previous hour's values
            
            soc_val = \
                BATTERY_SOC_BEGINNING_OF_YR \
                - (row[load_after_therm_gen_baseline_and_vre_prod_colname] \
                / battery_efficiency) \
                + row[curtailment_after_therm_gen_baseline_and_vre_colname]
            
            batt_curtailment_val = BATTERY_SOC_BEGINNING_OF_YR \
                - row[load_after_therm_gen_baseline_and_vre_prod_colname] \
                + row[curtailment_after_therm_gen_baseline_and_vre_colname] \
                - soc_val
            
            discharge_val = BATTERY_SOC_BEGINNING_OF_YR \
                - row[load_after_therm_gen_baseline_and_vre_prod_colname] \
                + row[curtailment_after_therm_gen_baseline_and_vre_colname] \
                - soc_val \
                - batt_curtailment_val
                
        else: # Subsequent rows. Has values for previous hour
            
            prev_soc = decorated_df.loc[idx - 1, battery_soc_colname]
            load_after_therm_gen_baseline_and_vre_prod = row[load_after_therm_gen_baseline_and_vre_prod_colname]
            curtailment_after_therm_gen_baseline_and_vre = row[curtailment_after_therm_gen_baseline_and_vre_colname]
            
            soc_val = prev_soc \
                - (load_after_therm_gen_baseline_and_vre_prod / battery_efficiency)\
                + curtailment_after_therm_gen_baseline_and_vre
            
            soc_val = soc_val if soc_val > BATTERY_MIN_SOC_MWH else BATTERY_MIN_SOC_MWH
            soc_val = soc_val if soc_val < BATTERY_CAPACITY_MWH else BATTERY_CAPACITY_MWH
            
            batt_curtailment_val = prev_soc \
                - load_after_therm_gen_baseline_and_vre_prod \
                + curtailment_after_therm_gen_baseline_and_vre \
                - soc_val
            
            batt_curtailment_val = batt_curtailment_val if batt_curtailment_val > 0 else 0
            
            discharge_val = (prev_soc - soc_val) if (prev_soc - soc_val) > 0 else 0
        
        decorated_df.at[idx, battery_soc_colname] = soc_val
        decorated_df.at[idx, battery_curtailment_after_charge_mw_colname] = batt_curtailment_val
        decorated_df.at[idx, battery_discharge_mw_colname] = discharge_val

    decorated_df[battery_soc_colname] = decorated_df[battery_soc_colname].astype(float)
    decorated_df[battery_curtailment_after_charge_mw_colname] = decorated_df[battery_curtailment_after_charge_mw_colname].astype(float)
    decorated_df[battery_discharge_mw_colname] = decorated_df[battery_discharge_mw_colname].astype(float)

    decorated_df[thermal_gen_delta_from_baseline_colname] = decorated_df.apply(
        lambda x: -x[battery_curtailment_after_charge_mw_colname] \
            if x[battery_curtailment_after_charge_mw_colname] < x[therm_gen_baseline_colname] - THERMAL_MIN_GEN_MW  \
            else THERMAL_MIN_GEN_MW - x[therm_gen_baseline_colname], axis=1
    )
    
    return decorated_df

In [14]:
battery_config = {
    battery_capacity_mwh_config_prop : BATTERY_CAPACITY_MWH,
    battery_power_mw_config_prop : BATTERY_POWER_MW,
    battery_efficiency_config_prop : BATTERY_EFFICIENCY
}

In [15]:
battery_hourly_df = add_intermediate_battery_cols(final_load_and_vre_df, battery_config)

In [16]:
battery_hourly_df.head(24)

,date,load_before_transmission_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,hour,load_w_transmission_loss_and_load_scale,therm_gen_baseline,load_after_therm_gen_baseline,total_vre_gen_mw,load_after_therm_gen_baseline_and_vre_prod,curtailment_after_therm_gen_baseline_and_vre,battery_soc_mwh,battery_curtailment_after_charge_mw,battery_discharge_mw,thermal_gen_delta_from_baseline
0,2017-01-01 00:00:00,35.1,1.68,0.0000,1,1,1,0,35.1,31,4.1,1.6800,2.4200,0.0000,177.144286,0.345714,0.000000,-0.345714
1,2017-01-01 01:00:00,35.1,1.23,0.0000,1,1,1,1,35.1,31,4.1,1.2300,2.8700,0.0000,173.864286,0.410000,3.280000,-0.410000
2,2017-01-01 02:00:00,34.6,1.09,0.0000,1,1,1,2,34.6,31,3.6,1.0900,2.5100,0.0000,170.995714,0.358571,2.868571,-0.358571
3,2017-01-01 03:00:00,34.2,1.47,0.0000,1,1,1,3,34.2,31,3.2,1.4700,1.7300,0.0000,169.018571,0.247143,1.977143,-0.247143
4,2017-01-01 04:00:00,33.8,1.12,0.0000,1,1,1,4,33.8,31,2.8,1.1200,1.6800,0.0000,167.098571,0.240000,1.920000,-0.240000
5,2017-01-01 05:00:00,33.4,0.91,0.0000,1,1,1,5,33.4,31,2.4,0.9100,1.4900,0.0000,165.395714,0.212857,1.702857,-0.212857
6,2017-01-01 06:00:00,33.3,1.84,0.0000,1,1,1,6,33.3,31,2.3,1.8400,0.4600,0.0000,164.870000,0.065714,0.525714,-0.065714
7,2017-01-01 07:00:00,33.8,1.59,6.8978,1,1,1,7,33.8,31,2.8,8.4878,0.0000,5.6878,170.557800,0.000000,0.000000,-0.000000
8,2017-01-01 08:00:00,34.7,1.18,21.3585,1,1,1,8,34.7,31,3.7,22.5385,0.0000,18.8385,189.396300,0.000000,0.000000,-0.000000
9,2017-01-01 09:00:00,34.7,0.78,27.9603,1,1,1,9,34.7,31,3.7,28.7403,0.0000,25.0403,214.436600,0.000000,0.000000,-0.000000


In [17]:
battery_final_thermal_prod_mw_colname = 'battery_final_thermal_prod_mw'
battery_final_remaining_load_mw_colname = 'battery_final_remaining_load_mw' 
battery_vre_to_grid_colname = 'battery_vre_to_grid'
battery_to_load_mw_colname = 'battery_to_load_mw'
battery_final_vre_to_battery_colname = 'battery_vre_to_battery'
battery_final_curtailed_vre_mw = 'battery_curtailed_vre_mw'
cruise_ships_boolean_colname = 'cruise_ships_boolean'
battery_curtail_available_for_cruise_colname = 'battery_curtail_available_for_cruise'

def add_optimized_battery_columns(df, config):
    
    decorated_df = df.copy()
    
    decorated_df[battery_final_thermal_prod_mw_colname] = \
        decorated_df[therm_gen_baseline_colname] + decorated_df[thermal_gen_delta_from_baseline_colname]
    
    decorated_df[battery_final_remaining_load_mw_colname] = \
        decorated_df[load_w_transmission_loss_and_load_scale_colname] - decorated_df[battery_final_thermal_prod_mw_colname]
    
    decorated_df[battery_vre_to_grid_colname] = decorated_df.apply(
        lambda x: x[battery_final_remaining_load_mw_colname] \
            if x[battery_final_remaining_load_mw_colname] < x[total_vre_gen_colname] \
            else x[total_vre_gen_colname]
        , axis=1
    )

    decorated_df[battery_to_load_mw_colname] = decorated_df.apply(
        lambda x: x[battery_final_remaining_load_mw_colname] - x[battery_vre_to_grid_colname] 
            if x[battery_final_remaining_load_mw_colname] - x[battery_vre_to_grid_colname] > 0 \
            else 0
        ,axis=1
    )
    decorated_df[battery_to_load_mw_colname] = decorated_df[battery_to_load_mw_colname].astype(float)
    
    decorated_df[battery_final_curtailed_vre_mw] = \
        decorated_df[battery_curtailment_after_charge_mw_colname] + decorated_df[thermal_gen_delta_from_baseline_colname]
    
    decorated_df[battery_final_vre_to_battery_colname] = None
    decorated_df.reset_index()
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            vre_to_battery_val = 0 if row[battery_final_remaining_load_mw_colname] - row[battery_vre_to_grid_colname] > 0\
                else BATTERY_CAPACITY_MWH - row[battery_soc_colname]
            
        else:
            vre_to_battery_val = row[battery_discharge_mw_colname] \
                if row[battery_final_remaining_load_mw_colname] - row[battery_vre_to_grid_colname] <= 0 \
                else 0 
        
        decorated_df.at[idx, battery_final_vre_to_battery_colname] = vre_to_battery_val
    
    decorated_df[cruise_ships_boolean_colname] = decorated_df.apply( 
        lambda x: True if 8 <= int(x[hour_colname]) <= 16 else False
        , axis=1)
    
    decorated_df[battery_curtail_available_for_cruise_colname] = decorated_df.apply( 
        lambda x: x[battery_final_curtailed_vre_mw] if x[cruise_ships_boolean_colname] == True else 0
        , axis=1)
    
    return decorated_df

In [18]:
battery_hourly_df = add_optimized_battery_columns(battery_hourly_df, battery_config)

In [19]:
battery_hourly_df

,date,load_before_transmission_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,hour,load_w_transmission_loss_and_load_scale,therm_gen_baseline,...,battery_discharge_mw,thermal_gen_delta_from_baseline,battery_final_thermal_prod_mw,battery_final_remaining_load_mw,battery_vre_to_grid,battery_to_load_mw,battery_curtailed_vre_mw,battery_vre_to_battery,cruise_ships_boolean,battery_curtail_available_for_cruise
0,2017-01-01 00:00:00,35.1,1.68,0.0,1,1,1,0,35.1,31,...,0.000000,-0.345714,30.654286,4.445714,1.68,2.765714,0.0,0,False,0.0
1,2017-01-01 01:00:00,35.1,1.23,0.0,1,1,1,1,35.1,31,...,3.280000,-0.410000,30.590000,4.510000,1.23,3.280000,0.0,0,False,0.0
2,2017-01-01 02:00:00,34.6,1.09,0.0,1,1,1,2,34.6,31,...,2.868571,-0.358571,30.641429,3.958571,1.09,2.868571,0.0,0,False,0.0
3,2017-01-01 03:00:00,34.2,1.47,0.0,1,1,1,3,34.2,31,...,1.977143,-0.247143,30.752857,3.447143,1.47,1.977143,0.0,0,False,0.0
4,2017-01-01 04:00:00,33.8,1.12,0.0,1,1,1,4,33.8,31,...,1.920000,-0.240000,30.760000,3.040000,1.12,1.920000,0.0,0,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,45.5,6.77,0.0,12,31,365,19,45.5,31,...,8.834286,-1.104286,29.895714,15.604286,6.77,8.834286,0.0,0,False,0.0
8756,2017-12-31 20:00:00,46.5,6.21,0.0,12,31,365,20,46.5,31,...,10.617143,-1.327143,29.672857,16.827143,6.21,10.617143,0.0,0,False,0.0
8757,2017-12-31 21:00:00,45.0,8.73,0.0,12,31,365,21,45.0,31,...,6.022857,-0.752857,30.247143,14.752857,8.73,6.022857,0.0,0,False,0.0
8758,2017-12-31 22:00:00,41.7,7.23,0.0,12,31,365,22,41.7,31,...,3.965714,-0.495714,30.504286,11.195714,7.23,3.965714,0.0,0,False,0.0


# Business as Usual

In [20]:
thermal_gen_total_mw_colname = 'gen_thermal_total_mw'
thermal_gen_total_kwh_colname = 'thermal_gen_total_kwh'

bau_df = final_load_and_vre_df.copy()
bau_df[thermal_gen_total_mw_colname] = bau_df[load_w_transmission_loss_and_load_scale_colname] - bau_df[total_vre_gen_colname]
bau_df[thermal_gen_total_kwh_colname] = bau_df[thermal_gen_total_mw_colname] * 1000

In [21]:
# Legacy generation options #########################
NOMINAL_HEAT_RATE_BTU_COMBUSTION = 17680
NOMINAL_HEAT_RATE_BTU_RICE = 9827
#####################################################

therm_rice_config = {
    "o_and_m" : .0105, # in cents / kwh
    "nominal_heat_rate_btu" : NOMINAL_HEAT_RATE_BTU_RICE, # in BTU / kWh
    "fuel_cost" : 2.50, # in $ / gallon
    "nominal_heat_rate_gal" : NOMINAL_HEAT_RATE_BTU_RICE / 139_000,
    "column_prepend" : "rice"
}

therm_combustion_config = {
    "o_and_m" : .0046, # in $ / kWh
    "nominal_heat_rate_btu" : NOMINAL_HEAT_RATE_BTU_COMBUSTION, # in BTU / kWh 
    "fuel_cost" : 2.50, # in $ / gallon
    "nominal_heat_rate_gal" : NOMINAL_HEAT_RATE_BTU_COMBUSTION / 139_000,
    "column_prepend" : "combustion"
}


In [22]:
def calc_bau_cols(df, config):
    
    decorated_df = df.copy()

    col_prepend = config['column_prepend']
    o_and_m = config['o_and_m']
    nominal_heat_rate_gal = config['nominal_heat_rate_gal']
    fuel_cost = config['fuel_cost']
    
    decorated_df['{}_fuel_cost'.format(col_prepend)] = decorated_df[thermal_gen_total_kwh_colname] * nominal_heat_rate_gal * fuel_cost
    decorated_df['{}_o_and_m_cost'.format(col_prepend)] = decorated_df[thermal_gen_total_kwh_colname] * o_and_m
    decorated_df['{}_total_cost'.format(col_prepend)] = decorated_df['{}_o_and_m_cost'.format(col_prepend)] + decorated_df['{}_fuel_cost'.format(col_prepend)]
    
    return decorated_df

In [23]:
bau_df = calc_bau_cols(bau_df, therm_rice_config)
bau_df = calc_bau_cols(bau_df, therm_combustion_config)
bau_df

,date,load_before_transmission_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,hour,load_w_transmission_loss_and_load_scale,therm_gen_baseline,...,load_after_therm_gen_baseline_and_vre_prod,curtailment_after_therm_gen_baseline_and_vre,gen_thermal_total_mw,thermal_gen_total_kwh,rice_fuel_cost,rice_o_and_m_cost,rice_total_cost,combustion_fuel_cost,combustion_o_and_m_cost,combustion_total_cost
0,2017-01-01 00:00:00,35.1,1.68,0.0,1,1,1,0,35.1,31,...,2.42,0.0,33.42,33420.0,5906.804676,350.910,6257.714676,10627.079137,153.732,10780.811137
1,2017-01-01 01:00:00,35.1,1.23,0.0,1,1,1,1,35.1,31,...,2.87,0.0,33.87,33870.0,5986.339748,355.635,6341.974748,10770.172662,155.802,10925.974662
2,2017-01-01 02:00:00,34.6,1.09,0.0,1,1,1,2,34.6,31,...,2.51,0.0,33.51,33510.0,5922.711691,351.855,6274.566691,10655.697842,154.146,10809.843842
3,2017-01-01 03:00:00,34.2,1.47,0.0,1,1,1,3,34.2,31,...,1.73,0.0,32.73,32730.0,5784.850899,343.665,6128.515899,10407.669065,150.558,10558.227065
4,2017-01-01 04:00:00,33.8,1.12,0.0,1,1,1,4,33.8,31,...,1.68,0.0,32.68,32680.0,5776.013669,343.140,6119.153669,10391.769784,150.328,10542.097784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,45.5,6.77,0.0,12,31,365,19,45.5,31,...,7.73,0.0,38.73,38730.0,6845.318525,406.665,7251.983525,12315.582734,178.158,12493.740734
8756,2017-12-31 20:00:00,46.5,6.21,0.0,12,31,365,20,46.5,31,...,9.29,0.0,40.29,40290.0,7121.040108,423.045,7544.085108,12811.640288,185.334,12996.974288
8757,2017-12-31 21:00:00,45.0,8.73,0.0,12,31,365,21,45.0,31,...,5.27,0.0,36.27,36270.0,6410.526799,380.835,6791.361799,11533.338129,166.842,11700.180129
8758,2017-12-31 22:00:00,41.7,7.23,0.0,12,31,365,22,41.7,31,...,3.47,0.0,34.47,34470.0,6092.386511,361.935,6454.321511,10960.964029,158.562,11119.526029


# Hydrogen-specific analysis

In [24]:
h2_df = final_load_and_vre_df.copy()
h2_df.head(24)

,date,load_before_transmission_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,hour,load_w_transmission_loss_and_load_scale,therm_gen_baseline,load_after_therm_gen_baseline,total_vre_gen_mw,load_after_therm_gen_baseline_and_vre_prod,curtailment_after_therm_gen_baseline_and_vre
0,2017-01-01 00:00:00,35.1,1.68,0.0000,1,1,1,0,35.1,31,4.1,1.6800,2.4200,0.0000
1,2017-01-01 01:00:00,35.1,1.23,0.0000,1,1,1,1,35.1,31,4.1,1.2300,2.8700,0.0000
2,2017-01-01 02:00:00,34.6,1.09,0.0000,1,1,1,2,34.6,31,3.6,1.0900,2.5100,0.0000
3,2017-01-01 03:00:00,34.2,1.47,0.0000,1,1,1,3,34.2,31,3.2,1.4700,1.7300,0.0000
4,2017-01-01 04:00:00,33.8,1.12,0.0000,1,1,1,4,33.8,31,2.8,1.1200,1.6800,0.0000
5,2017-01-01 05:00:00,33.4,0.91,0.0000,1,1,1,5,33.4,31,2.4,0.9100,1.4900,0.0000
6,2017-01-01 06:00:00,33.3,1.84,0.0000,1,1,1,6,33.3,31,2.3,1.8400,0.4600,0.0000
7,2017-01-01 07:00:00,33.8,1.59,6.8978,1,1,1,7,33.8,31,2.8,8.4878,0.0000,5.6878
8,2017-01-01 08:00:00,34.7,1.18,21.3585,1,1,1,8,34.7,31,3.7,22.5385,0.0000,18.8385
9,2017-01-01 09:00:00,34.7,0.78,27.9603,1,1,1,9,34.7,31,3.7,28.7403,0.0000,25.0403


In [25]:

# Hydrogen storage ##################################

# physical constants
H2_DENSITY_STP_KG_PER_M3 = 0.0813                # Hydrogen Tools. (2020). Hydrogen Density at different temperatures and pressures. Retrieved from https://h2tools.org/hyarc/hydrogen-data/hydrogen-density-different-temperatures-and-pressures
H2_LHV_MJ_PER_KG = 119.93                        # Hydrogen Fuel Cell Engines and Related Technologies. (2001). Module 1: Hydrogen Properties. Hydrogen Fuel Cell Engines, 41. 
H2_ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 = 1.8 # Elimelech, M., & Phillip, W. A. (2011). The future of seawater desalination: Energy, technology, and the environment. Science, 333(6043), 712–717. https://doi.org/10.1126/science.1200488
# electrolyzer
H2_ELEC_EFF_ELECTROLYZER_WRT_LHV_OF_H2 = .601    # Peterson, D., Vickers, J., & DeSantis, D. (2020). Hydrogen Production Cost From PEM Electrolysis - 2019. Retrieved from papers2://publication/uuid/5A951C39-B058-4DC0-B7C7-F8BA486F0706
H2_ELEC_EFF_ELECTROLYSIS_KWH_PER_KG = 55.4307635 # ^^
H2_TOTAL_UNINSTALLED_COST_USD_PER_KW = 460       # ^^
H2_INSTALLATION_COST_PERC_UNINSTALLED_CAPITAL_COST = .12 # ^^
H2_TAP_WATER_CONSUMPTION_L_PER_M3 = 1.4                  # Thomas, D. (2019). State of play and developments of power-to-hydrogen technologies. ETIP Wind Workship on Wind-to-Hydrogen, (February).
H2_MAX_ELECTROLYZER_POWER_INPUT_MW = 41
# fuel cell
H2_ELEC_EFF_FUEL_CELL_WRT_LHV_OF_H2 = .60        # Fuel Cell Technology Office. (2015). Fuel Cells Fact Sheet. https://doi.org/10.1007/978-1-349-04829-8_3      
H2_ELEC_EFF_FUEL_CELL_KWH_PER_KG = ((H2_LHV_MJ_PER_KG * H2_ELEC_EFF_FUEL_CELL_WRT_LHV_OF_H2) / 3600) * 1000
H2_TOTAL_UNINSTALLED_COST_USD_PER_KW = 180       # Satyapal, S., Cell, F., & Office, T. (2018). U . S . Department of Energy Hydrogen and Fuel Cell Technology Overview The Versatility of Fuel Cell Technologies.
# compression
H2_COMPRESSION_CAPEX_KG_H2_PER_DAY = 891 # (HyET Hydrogen. (2020). Electrochemical compression - Technology and performance. Retrieved from http://hyet.nl/hydrogen/technology-and-performance/)
H2_OPEX_USD_PER_KG = 1.08
H2_ENERGY_CONSUMPTION_KWH_PER_KG = 4
# storage
H2_PUMPING_DEMAND_KWH_PER_KG = 0
H2_STORAGE_STARTING_KG = 55550
H2_MIN_RESERVE = 42071
H2_STORAGE_MAX_CAPACITY = 65000

H2_SYSTEM_EFFICIENCY_KWH_PER_KG = (H2_ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 * H2_TAP_WATER_CONSUMPTION_L_PER_M3 / H2_DENSITY_STP_KG_PER_M3 /1000) \
    + H2_ELEC_EFF_ELECTROLYSIS_KWH_PER_KG + H2_ENERGY_CONSUMPTION_KWH_PER_KG + H2_PUMPING_DEMAND_KWH_PER_KG


In [26]:
h2_production_kg_colname = 'h2_production_kg'
h2_surplus_gen_mw_colname = 'h2_surplus_gen_mw'
h2_h20_demand_m3_colname = 'h2_h20_demand_m3'
h2_compression_and_desal_demand_mw_colname = 'h2_compression_and_desal_demand_mw'
h2_electrolyzer_load_colname = 'h2_electrolyzer_load'
h2_fuel_cell_power_gen_mw_colname = 'h2_fuel_cell_power_gen_mw'
h2_fuel_cell_consumption_colname = 'h2_fuel_cell_consumption'
h2_storage_state_colname = 'h2_storage_state'
h2_production_after_constraints_colname = 'h2_production_after_constraints'
h2_combo_load_mw_colname = 'h2_combination_load_mw_colname'
h2_curtailed_vre_colname = 'h2_curtailed_vre'
h2_thermal_gen_delta_colname = 'h2_thermal_gen_delta'

def calc_intermediate_h2_cols(df):
    
    decorated_df = df.copy()
    
    decorated_df[h2_surplus_gen_mw_colname] = decorated_df.apply(
        lambda x: H2_MAX_ELECTROLYZER_POWER_INPUT_MW \
            if x[curtailment_after_therm_gen_baseline_and_vre_colname] > H2_MAX_ELECTROLYZER_POWER_INPUT_MW \
            else x[curtailment_after_therm_gen_baseline_and_vre_colname],
        axis=1
    )
        
    decorated_df[h2_production_kg_colname] = (decorated_df[h2_surplus_gen_mw_colname] * 1000) / H2_SYSTEM_EFFICIENCY_KWH_PER_KG
    
    decorated_df[h2_h20_demand_m3_colname] = (decorated_df[h2_production_kg_colname] / H2_DENSITY_STP_KG_PER_M3) * (H2_TAP_WATER_CONSUMPTION_L_PER_M3 / 1000)
    
    decorated_df[h2_compression_and_desal_demand_mw_colname] = \
        (decorated_df[h2_production_kg_colname] * H2_ENERGY_CONSUMPTION_KWH_PER_KG  / 1000) \
        + (decorated_df[h2_h20_demand_m3_colname] * H2_ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 / 1000)
    
    decorated_df[h2_electrolyzer_load_colname] = decorated_df[h2_production_kg_colname] \
        * H2_ELEC_EFF_ELECTROLYSIS_KWH_PER_KG / 1000
    
    decorated_df[h2_fuel_cell_power_gen_mw_colname] = decorated_df.apply(
        lambda x: x[load_after_therm_gen_baseline_and_vre_prod_colname] if x[h2_production_kg_colname] == 0 else 0, axis=1
    )
    
    decorated_df[h2_fuel_cell_consumption_colname] = (decorated_df[h2_fuel_cell_power_gen_mw_colname] / H2_ELEC_EFF_FUEL_CELL_KWH_PER_KG )* 1000 

    decorated_df[h2_storage_state_colname] = None
    decorated_df[h2_production_after_constraints_colname] = None
    
    decorated_df.reset_index()
    for idx, row in decorated_df.iterrows():
        if idx == 0:
            storage_state_val = H2_STORAGE_STARTING_KG + row[h2_production_kg_colname] - row[h2_fuel_cell_consumption_colname]
            h2_prod_after_constraints = 0
        else:
            
            prev_storage_state = decorated_df.loc[idx - 1, h2_storage_state_colname]
            fuel_cell_h2_consumption = row[h2_fuel_cell_consumption_colname]
            h2_prod_kg = row[h2_production_kg_colname]
            storage_state_sum = prev_storage_state + h2_prod_kg - fuel_cell_h2_consumption
            
            if H2_MIN_RESERVE < storage_state_sum < H2_STORAGE_MAX_CAPACITY:
                storage_state_val = storage_state_sum
            elif storage_state_sum > H2_STORAGE_MAX_CAPACITY:
                storage_state_val = H2_STORAGE_MAX_CAPACITY
            else:
                storage_state_val = H2_MIN_RESERVE
            
            h2_prod_after_constraints = storage_state_val - prev_storage_state \
                if row[h2_production_kg_colname] > 0 else 0
            
        decorated_df.at[idx, h2_storage_state_colname] = storage_state_val
        decorated_df.at[idx, h2_production_after_constraints_colname] = h2_prod_after_constraints
            

    decorated_df[h2_combo_load_mw_colname] = \
        (decorated_df[h2_production_after_constraints_colname] * H2_SYSTEM_EFFICIENCY_KWH_PER_KG )/ 1000 
    

    decorated_df[h2_storage_state_colname] = decorated_df[h2_storage_state_colname].astype(float)

    decorated_df[h2_curtailed_vre_colname] = None
    for idx, row in decorated_df.iterrows():
        if row[curtailment_after_therm_gen_baseline_and_vre_colname] > 0:
            val = round(row[curtailment_after_therm_gen_baseline_and_vre_colname] - row[h2_combo_load_mw_colname] , 5)
        else:
            val = 0
        decorated_df.at[idx, h2_curtailed_vre_colname]  = val
        
    decorated_df[h2_thermal_gen_delta_colname] = - decorated_df[h2_curtailed_vre_colname] 
    return decorated_df




In [27]:
h2_intermediate_df = calc_intermediate_h2_cols(h2_df)

In [28]:
h2_real_thermal_prod_colname = 'h2_real_thermal_prod'
h2_real_remaining_load_colname = 'h2_real_remaing_load'
h2_vre_to_grid_colname = 'h2_vre_to_grid'
h2_fuel_cell_to_grid_colname = 'h2_fuel_cell_to_grid'
h2_vre_to_h2_prod_colname = 'h2_vre_to_h2_prod'
h2_final_curtailed_vre_colname = 'h2_final_curtailed_vre'
h2_curtail_available_for_cruise_colname = 'h2_curtail_available_for_cruise'

def calc_optimized_h2_cols(df):
    
    decorated_df = df.copy()
    
    decorated_df[h2_real_thermal_prod_colname] = decorated_df.apply(
        lambda x: x[therm_gen_baseline_colname] + x[h2_thermal_gen_delta_colname] \
            if x[therm_gen_baseline_colname] + x[h2_thermal_gen_delta_colname] > THERMAL_MIN_GEN_MW \
            else THERMAL_MIN_GEN_MW,
        axis = 1
    )
    
    decorated_df[h2_real_remaining_load_colname] = decorated_df[load_w_transmission_loss_and_load_scale_colname] - decorated_df[h2_real_thermal_prod_colname]
    
    decorated_df[h2_vre_to_grid_colname] = decorated_df.apply(
        lambda x: x[total_vre_gen_colname] if x[h2_real_remaining_load_colname] > x[total_vre_gen_colname] else x[h2_real_remaining_load_colname],
        axis=1
    )
    
    decorated_df[h2_fuel_cell_to_grid_colname] = decorated_df.apply(
        lambda x: x[h2_real_remaining_load_colname] - x[h2_vre_to_grid_colname] \
            if x[h2_real_remaining_load_colname] - x[h2_vre_to_grid_colname] > 0 \
            else 0,
        axis=1
    )
    
    decorated_df[h2_vre_to_h2_prod_colname] = decorated_df.apply(
        lambda x: x[h2_combo_load_mw_colname] \
            if x[h2_real_remaining_load_colname] - x[h2_vre_to_grid_colname] <= 0 \
            else 0,
        axis=1
    )
    
    decorated_df[h2_final_curtailed_vre_colname] = decorated_df[total_vre_gen_colname] \
        - (decorated_df[h2_vre_to_grid_colname] \
        + decorated_df[h2_vre_to_h2_prod_colname])
    
    decorated_df[cruise_ships_boolean_colname] = decorated_df.apply( 
        lambda x: True if 8 <= int(x[hour_colname]) <= 16 else False
        , axis=1)
    
    decorated_df[h2_curtail_available_for_cruise_colname] = decorated_df.apply( 
        lambda x: round(x[h2_final_curtailed_vre_colname],5) if x[cruise_ships_boolean_colname] == True else 0
        , axis=1)
    
    return decorated_df

In [29]:
h2_final_df = calc_optimized_h2_cols(h2_intermediate_df)

In [30]:
h2_final_df

,date,load_before_transmission_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,hour,load_w_transmission_loss_and_load_scale,therm_gen_baseline,...,h2_curtailed_vre,h2_thermal_gen_delta,h2_real_thermal_prod,h2_real_remaing_load,h2_vre_to_grid,h2_fuel_cell_to_grid,h2_vre_to_h2_prod,h2_final_curtailed_vre,cruise_ships_boolean,h2_curtail_available_for_cruise
0,2017-01-01 00:00:00,35.1,1.68,0.0,1,1,1,0,35.1,31,...,0,0,31.0,4.1,1.68,2.42,0.0,0.0,False,0.0
1,2017-01-01 01:00:00,35.1,1.23,0.0,1,1,1,1,35.1,31,...,0,0,31.0,4.1,1.23,2.87,0.0,0.0,False,0.0
2,2017-01-01 02:00:00,34.6,1.09,0.0,1,1,1,2,34.6,31,...,0,0,31.0,3.6,1.09,2.51,0.0,0.0,False,0.0
3,2017-01-01 03:00:00,34.2,1.47,0.0,1,1,1,3,34.2,31,...,0,0,31.0,3.2,1.47,1.73,0.0,0.0,False,0.0
4,2017-01-01 04:00:00,33.8,1.12,0.0,1,1,1,4,33.8,31,...,0,0,31.0,2.8,1.12,1.68,0.0,0.0,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,45.5,6.77,0.0,12,31,365,19,45.5,31,...,0,0,31.0,14.5,6.77,7.73,0.0,0.0,False,0.0
8756,2017-12-31 20:00:00,46.5,6.21,0.0,12,31,365,20,46.5,31,...,0,0,31.0,15.5,6.21,9.29,0.0,0.0,False,0.0
8757,2017-12-31 21:00:00,45.0,8.73,0.0,12,31,365,21,45.0,31,...,0,0,31.0,14.0,8.73,5.27,0.0,0.0,False,0.0
8758,2017-12-31 22:00:00,41.7,7.23,0.0,12,31,365,22,41.7,31,...,0,0,31.0,10.7,7.23,3.47,0.0,0.0,False,0.0
